In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer

import re
import pickle
import os
import time

from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
df = preprocess_data(use_saved_file=True)

In [ ]:
#splitting the description into words
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

# Using the Google pretrained Word2Vec Model 
# If using for the first time, download and store in ../../data/ 
# (link: https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz)

EMBEDDING_FILE = '../../data/GoogleNews-vectors-negative300.bin.gz'
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# Training our corpus with Google Pretrained Model

google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(corpus)

#model.intersect_word2vec_format('./word2vec/GoogleNews-vectors-negative300.bin', lockf=1.0, binary=True)

google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)



In [ ]:
# Generate the average word2vec for the each set of anime reviews
def vectors(x: pd.DataFrame):

    # Creating a list for storing the vectors (description into vectors)
    global array_embeddings
    array_embeddings = []

    # Reading the each anime review set
    for line in df['cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model[word]
                else:
                    avgword2vec = avgword2vec + google_model[word]

        if avgword2vec is not None:
            avgword2vec = avgword2vec / count

            array_embeddings.append(avgword2vec)

# Calling the function vectors
vectors(df)

# finding cosine similarity for the vectors
cosine_similarities = cosine_similarity(array_embeddings, array_embeddings)